In [1]:
import src.config as config

In [2]:
from datetime import datetime, timedelta,timezone

def aware_utcnow():
    return datetime.now(timezone.utc)
aware_utcnow()

datetime.datetime(2024, 12, 21, 18, 18, 26, 8625, tzinfo=datetime.timezone.utc)

In [ ]:
import pandas as pd

#current_date = pd.Timestamp('2024-10-31 00:00:00')
current_date = pd.to_datetime('2024-10-30 23:00:00')

#current_date = pd.to_datetime(aware_utcnow()).floor('h')
print(f'{current_date=}')
print(type(current_date),f': {current_date=}')

current_date=Timestamp('2024-10-31 00:00:00')
<class 'pandas._libs.tslibs.timestamps.Timestamp'> : current_date=Timestamp('2024-10-31 00:00:00')


In [4]:
# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)
fetch_data_from

Timestamp('2024-10-03 00:00:00')

In [5]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    from_date_ = pd.Timestamp(from_date - timedelta(days=7*52)).replace(tzinfo=None)
    to_date_ = pd.Timestamp(to_date - timedelta(days=7*52)).replace(tzinfo=None)

    print(f'{from_date_=}, {to_date_=}')

    # Load data and normalize timezones for pickup_datetime
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides['pickup_datetime'] = pd.to_datetime(rides['pickup_datetime'], errors='coerce').dt.tz_localize(None)
    rides = rides[rides.pickup_datetime >= from_date_]

    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2['pickup_datetime'] = pd.to_datetime(rides_2['pickup_datetime'], errors='coerce').dt.tz_localize(None)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # Shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [6]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date_=Timestamp('2023-10-05 00:00:00'), to_date_=Timestamp('2023-11-02 00:00:00')
File 2023-10 was already in local storage
File 2023-11 was already in local storage


In [7]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 255/255 [00:00<00:00, 1470.75it/s]


In [8]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [9]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_ts'],
    event_time='pickup_ts',
)

2024-12-21 18:18:28,703 INFO: Initializing external client
2024-12-21 18:18:28,704 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-21 18:18:30,104 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


In [10]:
print(f"Total rows in ts_data: {len(ts_data)}")

Total rows in ts_data: 171360


In [11]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1192104/fs/1181777/fg/1393062


Uploading Dataframe: 100.00% |██████████| Rows 171360/171360 | Elapsed Time: 00:36 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1192104/jobs/named/time_series_hourly_feature_group_3_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_3_offline_fg_materialization', 'SPARK'),
 None)